In [26]:
import cfscrape 
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [51]:
def investing_com(url, currency='no'):
    today = pd.to_datetime(datetime.now().date())
    
    scraper = cfscrape.create_scraper() 
    scraped_data = scraper.get(url) 
    
    #responce = requests.get(url)
    #soup = BeautifulSoup(responce.text, 'html.parser')
    soup = BeautifulSoup(scraped_data.text, 'html.parser')

    data = soup.find_all('td', class_='datatable_cell__LJp3C', dir="ltr")
    time = soup.find_all('time')
    data_temp_list = [i.text for i in data]

    data_list = []

    for i in data_temp_list:
        i = i.replace('.', '')
        i = i.replace(',', '.')
        data_list.append(i)

    data = []

    for i in data_list:
        try:
            i = pd.to_numeric(i)
            data.append(i)
        except:
            pass

    if currency == 'yes':
        data = data[:23]

    else:
        pass

    time_temp_list = [i.text for i in time]
    time_temp_list = time_temp_list[1:]

    result = pd.DataFrame(data={'date': time_temp_list, 'price': data})
    result['date'] = pd.to_datetime(result['date'], dayfirst=True)

    result = result[result['date'] != today]

    result.drop_duplicates(inplace=True)
    result.reset_index(inplace=True, drop=True)

    return result

In [52]:
gdaxi = investing_com('https://ru.investing.com/indices/germany-30-historical-data')
imoex = investing_com('https://ru.investing.com/indices/mcx-historical-data')
spx = investing_com('https://ru.investing.com/indices/us-spx-500-historical-data')
ssek = investing_com('https://ru.investing.com/indices/shanghai-composite-historical-data')
urals = investing_com('https://ru.investing.com/commodities/crude-oil-urals-spot-futures-historical-data')
currency = investing_com('https://ru.investing.com/currencies/eur-usd-historical-data', currency='yes')